<a href="https://colab.research.google.com/github/vadym-ts/Purace_volcano_displacement/blob/main/Purace_volcano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Purace Volcano displacement, 2024-2025

Analysis is based on The PyGMTSAR InSAR library by Alexei Pechnicov (https://www.linkedin.com/in/alexey-pechnikov/).



$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

## Google Colab Installation

Install PyGMTSAR and required GMTSAR binaries (including SNAPHU)

In [ ]:
import platform, sys, os
if 'google.colab' in sys.modules:
    # install PyGMTSAR stable version from PyPI
    !{sys.executable} -m pip install -q pygmtsar
    # alternatively, install PyGMTSAR development version from GitHub
    #!{sys.executable} -m pip install -Uq git+https://github.com/mobigroup/gmtsar.git@pygmtsar2#subdirectory=pygmtsar
    # use PyGMTSAR Google Colab installation script to install binary dependencies
    # script URL: https://github.com/AlexeyPechnikov/pygmtsar/blob/pygmtsar2/pygmtsar/pygmtsar/data/google_colab.sh
    import importlib.resources as resources
    with resources.as_file(resources.files('pygmtsar.data') / 'google_colab.sh') as google_colab_script_filename:
        !sh {google_colab_script_filename}
    # enable custom widget manager as required by recent Google Colab updates
    from google.colab import output
    output.enable_custom_widget_manager()
    # initialize virtual framebuffer for interactive 3D visualization; required for headless environments
    import xvfbwrapper
    display = xvfbwrapper.Xvfb(width=800, height=600)
    display.start()

# specify GMTSAR installation path
PATH = os.environ['PATH']
if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + ':/usr/local/GMTSAR/bin/'
    %env PATH {PATH}

# display PyGMTSAR version
from pygmtsar import __version__
__version__

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from dask.distributed import Client
import dask

In [ ]:
# plotting modules
import pyvista as pv
# magic trick for white background
pv.set_plot_theme("document")
import panel
panel.extension(comms='ipywidgets')
panel.extension('vtk')
from contextlib import contextmanager
import matplotlib.pyplot as plt
@contextmanager
def mpl_settings(settings):
    original_settings = {k: plt.rcParams[k] for k in settings}
    plt.rcParams.update(settings)
    yield
    plt.rcParams.update(original_settings)
plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
%matplotlib inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

In [ ]:
from pygmtsar import S1, Stack, tqdm_dask, ASF, Tiles, XYZTiles

# recent Google Colab changes in early September 2025 broke Dask+Xarray NetCDF multithedded processing (again)
# workaround below disables multitheading when it does not work, degrading performance and increasing RAM usage.
if 'google.colab' in sys.modules:
    methods = {
        "load_dem":  "synchronous",
        "save_cube": "compute",
        "save_stack":"compute",
    }
    for m, kind in methods.items():
        if not hasattr(Stack, f"_{m}"):
            setattr(Stack, f"_{m}", getattr(Stack, m))
        def _make_wrapper(name, kind):
            orig = getattr(Stack, f"_{name}")
            if kind == "synchronous":
                def _wrapper(self, *args, **kwargs):
                    with dask.config.set(scheduler="synchronous"):
                        return orig(self, *args, **kwargs)
                return _wrapper
            elif kind == "compute":
                def _wrapper(self, *args, **kwargs):
                    if args:
                        return orig(self, args[0].compute() if hasattr(args[0], "compute") else args[0], *args[1:], **kwargs)
                    return orig(self, **kwargs)
                return _wrapper
            else:
                raise NotImplementedError(f"Unknown wrapper kind: {kind}")
        setattr(Stack, m, _make_wrapper(m, kind))

## Define Processing Parameters

In [ ]:
ORBIT    = 'D'

In [ ]:
WORKDIR      = f'raw_fogo_{ORBIT}'
DATADIR      = f'data_fogo_{ORBIT}'

In [ ]:
import os

# List contents of the DATADIR
print(f"Contents of {DATADIR}:")
if os.path.exists(DATADIR):
    for item in os.listdir(DATADIR):
        print(f"  - {item}")
else:
    print(f"The directory {DATADIR} does not exist.")

In [ ]:
# define DEM and landmask filenames inside data directory
DEM = f'{DATADIR}/dem.nc'


LANDMASK = f'{DATADIR}/landmask.nc'

In [ ]:
geojson = {
  "type": "Feature",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [-76.512863, 2.367566],
        [-76.507369, 2.249559],
        [-76.397491, 2.202216],
        [-76.279372, 2.251617],
        [-76.304781, 2.34767],
        [-76.413286, 2.400497],
        [-76.512863, 2.367566]
      ]
    ]
  },
  "properties": {}
}

AOI = gpd.GeoDataFrame.from_features([geojson])

## Download and Unpack Datasets

### Sentinel-1 SLC Search

In [ ]:
# find bursts for the first interval
bursts1 = ASF.search(AOI, startTime='2025-11-15', stopTime='2025-11-30', flightDirection=ORBIT)

# find bursts for the second interval
bursts2 = ASF.search(AOI, startTime='2025-12-07', stopTime='2025-12-14', flightDirection=ORBIT)

# combine the results of both searches
bursts = pd.concat([bursts1, bursts2], ignore_index=True)
bursts

In [ ]:
# print bursts
BURSTS = bursts.fileID.tolist()

print (f'Bursts defined: {len(BURSTS)}')
BURSTS

In [ ]:
# plot bursts
ASF.plot(bursts)
# plot AOI
AOI.plot(ax=plt.gca(), color='gold', edgecolor='darkgoldenrod', alpha=0.5, label='AOI')
# plot basemap
XYZTiles().download_googlesatellitehybrid(bursts.union_all().buffer(0.1), zoom=9).plot.imshow(ax=plt.gca())
plt.gca().set_aspect('equal')
plt.title('Area of Interest Location')
plt.show()

### Data Downloading

If the data directory is empty or doesn't exist, you'll need to download Sentinel-1 scenes from the Alaska Satellite Facility (ASF) datastore. Use your Earthdata Login credentials. If you don't have an Earthdata Login, you can create one at https://urs.earthdata.nasa.gov//users/new

You can also use pre-existing SLC scenes stored on your Google Drive, or you can copy them using a direct public link from iCloud Drive.

The credentials below are available at the time the notebook is validated.

In [ ]:
# Enter Your ASF User and Password.
# Set these variables to None and you will be prompted to enter your username and password below.
asf_username = ''
asf_password = ''

In [ ]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf = ASF(asf_username, asf_password)
# Optimized scene downloading from ASF - only the required subswaths and polarizations.
print(asf.download(DATADIR, BURSTS))


In [ ]:
# scan the data directory for SLC scenes and download missed orbits
S1.download_orbits(DATADIR, S1.scan_slc(DATADIR))

In [ ]:
Tiles().download_dem(AOI,filename=DEM, product = '3s').plot.imshow(cmap='cividis')

### Manually Download and Place EGM96 Geoid Grid

Since the automated download of `egm96.grd` seems to be failing, we will manually download it and place it in the expected directory. This file is essential for `pygmtsar` to perform height conversions.


In [ ]:
import os

EGM96_DIR = '/usr/local/share/gmtsar/EGM96/'
EGM96_FILE_URL = 'https://raw.githubusercontent.com/AlexeyPechnikov/pygmtsar/pygmtsar2/pygmtsar/pygmtsar/data/EGM96/egm96.grd'
EGM96_LOCAL_PATH = os.path.join(EGM96_DIR, 'egm96.grd')

# Create the directory if it doesn't exist
!mkdir -p {EGM96_DIR}

# Download the file using wget
if not os.path.exists(EGM96_LOCAL_PATH):
    print(f"Downloading {EGM96_FILE_URL} to {EGM96_LOCAL_PATH}...")
    !wget -q {EGM96_FILE_URL} -O {EGM96_LOCAL_PATH}
    if os.path.exists(EGM96_LOCAL_PATH):
        print("Download successful.")
    else:
        print("ERROR: Download failed. Please check the URL and your connection.")
else:
    print(f"EGM96 geoid grid already exists at {EGM96_LOCAL_PATH}.")

# Verify the file is now present
if os.path.exists(EGM96_LOCAL_PATH):
    print("EGM96 geoid grid is now confirmed to be in place.")
else:
    print("EGM96 geoid grid is still missing. There might be an issue with permissions or the download.")

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axes
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot the AOI
AOI.plot(ax=ax, color='gold', edgecolor='darkgoldenrod', alpha=0.5, label='AOI')

# Download and plot a satellite basemap for context
XYZTiles().download_googlesatellitehybrid(AOI.unary_union.buffer(0.1), zoom=9).plot.imshow(ax=ax)

# Set aspect ratio and add title/legend
ax.set_aspect('equal')
ax.set_title('Area of Interest (AOI) on Satellite Map')
ax.legend()
plt.show()

In [ ]:
from IPython.display import display as ipython_display
import json

# Display the geojson in a readable format
ipython_display(json.dumps(geojson, indent=2))

In [ ]:
# download land mask 1 arc-second
landmask_data = Tiles().download_landmask(AOI, filename=LANDMASK).fillna(0)
landmask_data.plot.imshow(cmap='binary_r')

## Run Local Dask Cluster

Launch Dask cluster for local and distributed multicore computing. That's possible to process terabyte scale Sentinel-1 SLC datasets on Apple Air 16 GB RAM.

In [ ]:
# simple Dask initialization
if 'client' in globals():
    client.close()
client = Client()
client

## Init SBAS

Search recursively for measurement (.tiff) and annotation (.xml) and orbit (.EOF) files in the DATA directory. It can be directory with full unzipped scenes (.SAFE) subdirectories or just a directory with the list of pairs of required .tiff and .xml files (maybe pre-filtered for orbit, polarization and subswath to save disk space). If orbit files and DEM are missed these will be downloaded automatically below.

### Select Original Secenes and Orbits

Use filters to find required subswath, polarization and orbit in original scenes .SAFE directories in the data directory.

In [ ]:
scenes = S1.scan_slc(DATADIR)

In [ ]:
sbas  = Stack(WORKDIR, drop_if_exists=True).set_scenes(scenes).set_reference('2025-11-16')
sbas.to_dataframe()

In [ ]:
sbas.plot_scenes(AOI=AOI, aspect='equal')

## Reframe Scenes (Optional)

Stitch sequential scenes and crop the subswath to a smaller area for faster processing when the full area is not needed.

In [ ]:
sbas.compute_reframe()

In [ ]:
sbas.plot_scenes(AOI=AOI, aspect='equal')

### Load DEM

The function below loads DEM from file or Xarray variable and converts heights to ellipsoidal model using EGM96 grid.

In [ ]:
# define the area of interest (AOI) to speedup the processing
#xr.open_dataset(DEM)['z'].to_netcdf('dem_z.nc'); sbas.load_dem('dem_z.nc', AOI)
sbas.load_dem(DEM, AOI)
sbas.load_landmask(LANDMASK)

In [ ]:
import os

EGM96_PATH = '/usr/local/share/gmtsar/EGM96/egm96.grd'

if os.path.exists(EGM96_PATH):
    print(f"EGM96 geoid grid found at: {EGM96_PATH}")
else:
    print(f"WARNING: EGM96 geoid grid NOT found at: {EGM96_PATH}")
    print("This file is critical for DEM loading. It should be downloaded by the pygmtsar installation script.")
    print("If it's missing, you may need to manually download it or investigate why the installation script failed.")


In [ ]:
sbas.plot_scenes(AOI=AOI, aspect='equal')

## Align Images

In [ ]:
sbas.compute_align()
#sbas.df.head()

## Geocoding Transform

In [ ]:
sbas.compute_geocode(45.)

In [ ]:
sbas.plot_topo()
plt.savefig('Topography in Radar Coordinates.jpg')

## Interferogram

The code below is detailed for education reasons and can be more compact excluding optional arguments. See other PyGMTSAR examples for shorter version.

In [ ]:
# get the unique dates from the stack, sorted
unique_dates = sorted(sbas.to_dataframe().index.unique().tolist())
# get the reference date which was set earlier
reference_date = sbas.reference

# Create a list of pairs where the reference_date is the first element
# and each other unique date is the second element.
# This assumes a standard SBAS approach where all scenes are paired with the reference.
pairs = []
for date in unique_dates:
    if date != reference_date:
        pairs.append([reference_date, date])
pairs = [['2025-11-16', '2025-12-10']]
pairs

In [ ]:
# load radar topography
topo = sbas.get_topo()
# load Sentinel-1 data
data = sbas.open_data()
# Gaussian filtering 90m cut-off wavelength with multilooking 3x12 on Sentinel-1 intensity
intensity = sbas.multilooking(np.square(np.abs(data)), wavelength=90, coarsen=(3,12))
# calculate phase difference with topography correction
phase = sbas.phasediff(pairs, data, topo)
# Gaussian filtering 90m cut-off wavelength with multilooking
phase = sbas.multilooking(phase, wavelength=90, coarsen=(3,12))
# correlation on 3x12 multilooking data
corr = sbas.correlation(phase, intensity)
# Goldstein filter in 32 pixel patch size on square grid cells produced using 1:4 range multilooking
phase_goldstein = sbas.goldstein(phase, corr, 16)
# convert complex phase difference to interferogram
intf = sbas.interferogram(phase_goldstein)
# materialize for a single interferogram
tqdm_dask(result := dask.persist(intf[0], corr[0]), desc='Compute Phase and Correlation')
# unpack results
intf, corr = result

In [ ]:
# geocode
intf_ll = sbas.ra2ll(intf)
corr_ll = sbas.ra2ll(corr)
dem = sbas.get_dem().interp_like(intf_ll).where(np.isfinite(intf_ll))
landmask_ll = sbas.get_landmask().interp_like(intf_ll)

In [ ]:
sbas.plot_landmask(landmask_ll, aspect='equal')

In [ ]:
sbas.plot_interferogram(intf_ll.where(landmask_ll), aspect='equal')
plt.savefig('Phase, [rad].jpg')

In [ ]:
sbas.plot_topo(dem.where(landmask_ll), aspect='equal')

In [ ]:
sbas.plot_correlation(corr_ll.where(landmask_ll), aspect='equal')
plt.savefig('Correlation.jpg')

In [ ]:
sbas.export_vtk(intf_ll.where(landmask_ll), 'intf', mask='auto')

In [ ]:
# build interactive 3D plot
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(pv.read('intf.vtk').scale([1, 1, 0.00002], inplace=True), scalars='phase', cmap='turbo', ambient=0.1, show_scalar_bar=True)
plotter.show_axes()
plotter.show(screenshot='3D Interferogram.png', jupyter_backend='panel', return_viewer=True)
plotter.add_title(f'Interactive Interferogram on DEM', font_size=32)
plotter._on_first_render_request()
panel.panel(
    plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
    enable_keybindings=False, sizing_mode='stretch_width', min_height=600
)

## Unwrapping

In [ ]:
# mask low-coherence areas using threshold value 0.075
tqdm_dask(unwrap := sbas.unwrap_snaphu(intf.where(corr>=0.075), corr).persist(),
          desc='SNAPHU Unwrapping')
# apply simplest detrending
unwrap['phase'] = unwrap.phase - unwrap.phase.mean()

In [ ]:
# geocode to geographic coordinates and crop empty borders
unwrap_ll = sbas.ra2ll(unwrap.phase)

In [ ]:
sbas.plot_phase(unwrap_ll.where(landmask_ll), caption='Unwrapped Phase\nGeographic Coordinates, [rad]', quantile=[0.02, 0.98], aspect='equal')
plt.savefig('Unwrapped Phase Geographic Coordinates, [rad].jpg')

In [ ]:
sbas.export_vtk(unwrap_ll.where(landmask_ll), 'unwrap', mask='auto')

In [ ]:
# build interactive 3D plot
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(pv.read('unwrap.vtk').scale([1, 1, 0.00002], inplace=True), scalars='phase', cmap='turbo', ambient=0.1, show_scalar_bar=True)
plotter.show_axes()
plotter.show(screenshot='3D Unwrapped Interferogram.png', jupyter_backend='panel', return_viewer=True)
plotter.add_title(f'Interactive Unwrapped Interferogram on DEM', font_size=32)
plotter._on_first_render_request()
panel.panel(
    plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
    enable_keybindings=False, sizing_mode='stretch_width', min_height=600
)

## LOS Displacement

In [ ]:
# geocode to geographic coordinates and crop empty borders
los_disp_mm_ll = sbas.los_displacement_mm(unwrap_ll)

In [ ]:
sbas.plot_displacement(los_disp_mm_ll.where(landmask_ll), caption='LOS Displacement Nov 16/ Dec 10 2025, Purace volcano \nGeographic Coordinates, [mm]', quantile=[0.01, 0.99], aspect='equal', cbar_label='Line-of-sight displacement, mm')
plt.savefig('LOS Displacement Geographic Coordinates, [mm].jpg')

In [ ]:
sbas.export_vtk(los_disp_mm_ll.where(landmask_ll), 'los', mask='auto')

In [ ]:
# build interactive 3D plot

# Calculate the color limits from the unwrapped phase for consistency

clim_min, clim_max = los_disp_mm_ll.quantile([0.01, 0.99]).values

plotter = pv.Plotter(notebook=True)
plotter.add_mesh(pv.read('los.vtk').scale([1, 1, 0.00002], inplace=True), scalars='los', cmap='turbo', ambient=0.1, show_scalar_bar=False, clim=[clim_min, clim_max])
plotter.show_axes()
plotter.show(screenshot='3D LOS Displacement.png', jupyter_backend='panel', return_viewer=True)
plotter.add_title(f'Interactive LOS Displacement on DEM', font_size=32)
plotter._on_first_render_request()
panel.panel(
    plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
    enable_keybindings=False, sizing_mode='stretch_width', min_height=600
)

In [ ]:
# compare to the top-right image in the 2x2 plot above
sbas.plot_interferogram(intf_ll.where(landmask_ll), caption=f'PyGMTSAR Interferogram, [rad]\nS1A Desc {pairs[0][0]} - {pairs[0][1]}', aspect='equal')
plt.savefig('PyGMTSAR Interferogram, [rad].jpg')

In [ ]:
sbas.export_geotiff(intf_ll.where(landmask_ll), 'tbilisi.tif')
print('done')
